# Homework 2: Discovery of Frequent Itemsets and Association Rules

In [1]:
import pandas as pd
#Please install mlxtend to import these
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

#Dataset downloaded from https://www.kaggle.com/carrie1/ecommerce-data
dataset=pd.read_csv('data.csv', encoding='cp1252')

In [2]:
r,c = dataset.shape
print ("Number of rows: " + str(r))
dataset.head()


Number of rows: 541909


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


# Preprocessing

In [3]:
#Removing spaces from the beginning and end of the descriptions
dataset['Description'] = dataset['Description'].str.strip()

#Dropping the rows that have no invoice date
dataset.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
dataset['InvoiceNo'] = dataset['InvoiceNo'].astype('str')
dataset = dataset[~dataset['InvoiceNo'].str.contains('C')]

r,c = dataset.shape
print ("Number of rows: " + str(r))
dataset.head()
#Looks like no change - no worries just being careful

Number of rows: 532621


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
#Checking the number of unique transactions in 'Country' column
#dataset.Country.unique()
transactions_country = dataset.drop_duplicates().groupby('Country')['Quantity'].count()
print(transactions_country)

Country
Australia                 1184
Austria                    398
Bahrain                     18
Belgium                   2031
Brazil                      32
Canada                     151
Channel Islands            747
Cyprus                     603
Czech Republic              25
Denmark                    380
EIRE                      7883
European Community          60
Finland                    685
France                    8393
Germany                   9027
Greece                     145
Hong Kong                  280
Iceland                    182
Israel                     292
Italy                      758
Japan                      321
Lebanon                     45
Lithuania                   35
Malta                      112
Netherlands               2363
Norway                    1072
Poland                     330
Portugal                  1492
RSA                         58
Saudi Arabia                 9
Singapore                  222
Spain                     2480


In [5]:
#Consolidating the items into 1 transaction per row (so the items are columns)
#Also, only taking the Germany data since it's big enough but not too big
basket = (dataset[dataset['Country'] == "Germany"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket.head(15)

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,...,YULETIDE IMAGES GIFT WRAP SET,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
#Listing the items - just in case it's of interest
for col in basket.columns: 
    print(col) 

10 COLOUR SPACEBOY PEN
12 COLOURED PARTY BALLOONS
12 IVORY ROSE PEG PLACE SETTINGS
12 MESSAGE CARDS WITH ENVELOPES
12 PENCIL SMALL TUBE WOODLAND
12 PENCILS SMALL TUBE RED RETROSPOT
12 PENCILS SMALL TUBE SKULL
12 PENCILS TALL TUBE POSY
12 PENCILS TALL TUBE RED RETROSPOT
12 PENCILS TALL TUBE SKULLS
12 PENCILS TALL TUBE WOODLAND
12 PINK HEN+CHICKS IN BASKET
12 RED ROSE PEG PLACE SETTINGS
16 PIECE CUTLERY SET PANTRY DESIGN
2 PICTURE BOOK EGGS EASTER BUNNY
2 PICTURE BOOK EGGS EASTER CHICKS
20 DOLLY PEGS RETROSPOT
200 BENDY SKULL STRAWS
200 RED + WHITE BENDY STRAWS
3 DRAWER ANTIQUE WHITE WOOD CABINET
3 HOOK HANGER MAGIC GARDEN
3 HOOK PHOTO SHELF ANTIQUE WHITE
3 PIECE SPACEBOY COOKIE CUTTER SET
3 RAFFIA RIBBONS 50'S CHRISTMAS
3 STRIPEY MICE FELTCRAFT
3 TIER CAKE TIN GREEN AND CREAM
3 TIER CAKE TIN RED AND CREAM
3 TIER SWEETHEART GARDEN SHELF
3 TRADITIONAl BISCUIT CUTTERS  SET
36 DOILIES DOLLY GIRL
36 FOIL HEART CAKE CASES
36 FOIL STAR CAKE CASES
36 PENCILS TUBE RED RETROSPOT
36 PENCILS TUBE S

RETROSPOT GIANT TUBE MATCHES
RETROSPOT HEART HOT WATER BOTTLE
RETROSPOT LAMP
RETROSPOT LARGE MILK JUG
RETROSPOT PADDED SEAT CUSHION
RETROSPOT PARTY BAG + STICKER SET
RETROSPOT RED WASHING UP GLOVES
RETROSPOT TEA SET CERAMIC 11 PC
REVOLVER WOODEN RULER
REX CASH+CARRY JUMBO SHOPPER
RIBBON REEL CHRISTMAS PRESENT
RIBBON REEL CHRISTMAS SOCK BAUBLE
RIBBON REEL FLORA + FAUNA
RIBBON REEL HEARTS DESIGN
RIBBON REEL LACE DESIGN
RIBBON REEL MAKING SNOWMEN
RIBBON REEL POLKADOTS
RIBBON REEL SNOWY VILLAGE
RIBBON REEL SOCKS AND MITTENS
RIBBON REEL STRIPES DESIGN
RIBBONS PURSE
RIDGED GLASS POSY VASE
RIDGED GLASS T-LIGHT HOLDER
RING OF ROSES BIRTHDAY CARD
ROBOT BIRTHDAY CARD
ROCKING HORSE GREEN CHRISTMAS
ROCKING HORSE RED CHRISTMAS
ROCOCO WALL MIRROR WHITE
ROLL WRAP 50'S CHRISTMAS
ROLL WRAP VINTAGE CHRISTMAS
ROLL WRAP VINTAGE SPOT
ROMANTIC IMAGES GIFT WRAP SET
ROMANTIC PINKS RIBBONS
ROSE CARAVAN DOORSTOP
ROSE COTTAGE KEEPSAKE BOX
ROSE DU SUD CUSHION COVER
ROSE DU SUD DRAWSTRING BAG
ROSE DU SUD OVEN GLOV

In [9]:
#Since we are only interested in the item regardless of its quantity per transaction
#we're only interested if it's bought or not
def bought_or_not(x):
    if x <= 0:        #Without this, it will return NaN if not >=
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(bought_or_not)
#For some reason POSTAGE'S StockCode value os "POST"
basket_sets.drop('POSTAGE', inplace=True, axis=1)
basket_sets.head(15)

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,...,YULETIDE IMAGES GIFT WRAP SET,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536840,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536983,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537201,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Frequent Itemsets and Association Rules

In [10]:
#Frequent itemsets
support=0.05 #The higher the support the less will be generated
frequent_itemsets = apriori(basket_sets, support, use_colnames=True)
print ('Frequent itemsets:\n\n', frequent_itemsets)

Frequent itemsets:

      support                                           itemsets
0   0.102845                           (6 RIBBONS RUSTIC CHARM)
1   0.070022                        (ALARM CLOCK BAKELIKE PINK)
2   0.065646                      (CHARLOTTE BAG APPLES DESIGN)
3   0.050328                     (CHILDRENS CUTLERY DOLLY GIRL)
4   0.061269                         (COFFEE MUG APPLES DESIGN)
5   0.063457                       (FAWN BLUE HOT WATER BOTTLE)
6   0.072210                                (GUMBALL COAT RACK)
7   0.056893                             (IVORY KITCHEN SCALES)
8   0.063457                            (JAM JAR WITH PINK LID)
9   0.091904                           (JAM MAKING SET PRINTED)
10  0.061269                                 (JUMBO BAG APPLES)
11  0.078775                          (JUMBO BAG RED RETROSPOT)
12  0.100656                       (JUMBO BAG WOODLAND ANIMALS)
13  0.061269                           (LUNCH BAG APPLE DESIGN)
14  0.078775       

In [11]:
#Association rules
metric="confidence" #Can use lift, or other metrics, as well
min_threshold = 0.1 #Again, the higher this is the less will pass
rules = association_rules(frequent_itemsets, metric, min_threshold)

print('Association rules:\n')
rules.head()

Association rules:



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(PLASTERS IN TIN WOODLAND ANIMALS),(PLASTERS IN TIN CIRCUS PARADE),0.137856,0.115974,0.067834,0.492063,4.242887,0.051846,1.740427
1,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.115974,0.137856,0.067834,0.584906,4.242887,0.051846,2.076984
2,(ROUND SNACK BOXES SET OF 4 FRUITS),(PLASTERS IN TIN CIRCUS PARADE),0.157549,0.115974,0.050328,0.319444,2.754455,0.032057,1.298977
3,(PLASTERS IN TIN CIRCUS PARADE),(ROUND SNACK BOXES SET OF 4 FRUITS),0.115974,0.157549,0.050328,0.433962,2.754455,0.032057,1.488330
4,(ROUND SNACK BOXES SET OF4 WOODLAND),(PLASTERS IN TIN CIRCUS PARADE),0.245077,0.115974,0.056893,0.232143,2.001685,0.028470,1.151290
